In [ ]:
import json
import tkinter as tk
from tkinter import filedialog

# Setup GUI
root = tk.Tk()
root.withdraw()  # Hide the main window

# Step 1: Select the theme file
input_path = filedialog.askopenfilename(title="Select Power BI Theme JSON File", filetypes=[("JSON files", "*.json")])
if not input_path:
    print("No file selected.")
    exit()

# Step 2: Load the theme file
with open(input_path, 'r', encoding='utf-8') as file:
    theme_data = json.load(file)

# Step 3: Build placeholder map
data_colors = theme_data.get("dataColors", [])
placeholder_map = {
    f"{{themecolor{i+1}}}": color.lstrip("#") for i, color in enumerate(data_colors)
}
placeholder_map.update({
    f"#{'{themecolor' + str(i+1) + '}'}": color.lstrip("#") for i, color in enumerate(data_colors)
})

# Step 4: Replace placeholders recursively
def replace_placeholders(obj):
    if isinstance(obj, dict):
        return {k: replace_placeholders(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [replace_placeholders(item) for item in obj]
    elif isinstance(obj, str):
        for placeholder, actual in placeholder_map.items():
            obj = obj.replace(placeholder, actual)
        return obj
    else:
        return obj

updated_theme = replace_placeholders(theme_data)

# Step 5: Prompt for save location
output_path = filedialog.asksaveasfilename(
    title="Save Updated Theme As",
    defaultextension=".json",
    filetypes=[("JSON files", "*.json")]
)
if not output_path:
    print("Save cancelled.")
    exit()

# Step 6: Save the updated theme
with open(output_path, 'w', encoding='utf-8') as file:
    json.dump(updated_theme, file, indent=2)

print(f"Updated theme saved to: {output_path}")